# Assignment 3 | Sentiment Analyisis

### Relevant Libraries

In [70]:
#libraries
import pandas as pd
import numpy as np
from prettytable import PrettyTable
import seaborn as sns
import matplotlib.pyplot as plt
import re
import regex
import string
from sklearn.feature_extraction.text import CountVectorizer

from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
#installing tweet-preprocessor
from tqdm.auto import tqdm
!pip install tweet-preprocessor --user
import preprocessor as p
from pathlib import Path

#emoji library
!pip install emoji --user
import emoji

# model save
import joblib

#split
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /home/hemant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/hemant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hemant/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
w_ = set(['never', 'no', 'nothing', 'nowhere', 'noone', 'none', 'not', 'havent', 'hasnt', 'hadnt', 'cant', 
                         'couldnt', 'shouldnt', 'wont', 'wouldnt', 'dont', 'doesnt', 'didnt', 'isnt', 'arent', 'aint'] )

In [ ]:
list('sdfnds')

In [ ]:
lits = ['.', ',', ':', ';', '!', '?']
count = 0
c = -1
x = f"I don't think I will enjoy it: it might be too spicy.".lower()
for w in x.split():
    if re.sub(r'[^\w\s]', '', w) in w_ or c > -1:
        c+=1
    if w[-1] in splits:
        count += c
        c = -1
count

In [ ]:
def negation(self, x):
    splits = ['.', ',', ':', ';', '!', '?']
    count = 0
    c = -1
    x = f"I don't think I will enjoy it: it might be too spicy.".lower()
    for w in x.split():
        if w in w_ or c > -1:
            print(w)
            c+=1
        if w[-1] in splits:
            count += c
            c = -1
    return count

In [73]:
class featurise(object):
    def __init__(self, x,y):
        super(featurise, self).__init__()
        self.seq = None
        self.x = x
        self.y = y
        ##########################################
        self.final_df = pd.DataFrame()
        self.final_df['tweets'] = x
        self.final_df['target'] = y
        ##########################################
        self.negation_w = set(['never', 'no', 'nothing', 'nowhere', 'noone', 'none', 'not', 'havent', 'hasnt', 'hadnt', 'cant', 
                         'couldnt', 'shouldnt', 'wont', 'wouldnt', 'dont', 'doesnt', 'didnt', 'isnt', 'arent', 'aint'] )
        self.stop_words = stopwords.words('english')
        self.stop_words.remove('not')

    def elongated_word(self, x):
        regex_ = re.compile(r"(.)\1{2}")
        if regex_.search(x): return True
        else: return False

    def emoji(self, x): # Doubt
        return len([1 for word in regex.findall(r'\X', x) if any(char in emoji.UNICODE_EMOJI for char in word)])
    
    def punctuation(self, x):
        punct_ = string.punctuation
        return len([1 for w in x if w in punct_])
    
    def all_caps(self, x):
        return len([1 for each in x if each.isupper()])

    def hashtag_count(self, x):
        return len([word for word in x.split() if word.startswith('#') and len(word)>1 ])
    
    def negation(self, x):
        flag = 0
        splits = ",.:;!?"
        final = []
        count = 0
#         x = f"I don't think I will enjoy it: it might be too spicy.".lower()
        for word in x.split():
            if flag ==1:
                for symb in splits:
                    if symb in word:
                        flag = 0
                        word_split = word.split(symb)
                        
                        if len(word_split[0])>0:
                        
                            count+=1
                        
                            if len(word_split[-1])>1:
                        
                                pass
                        else:
                            pass
                if flag==1:
                    
                    count+=1
                
            elif flag==0:
                
                pass

            clean = re.sub('[^A-Za-z0-9]+', '', word)
            if clean in self.negation_w:
                flag=1
        return count

    def pos_tag_feat(self, a):
        tagged_texts = pos_tag_sents(map(word_tokenize, a))

        df = pd.DataFrame(a)
        df['pos'] = tagged_texts
        pos_vocab=set()
        for pos in tagged_texts:
            for each in pos:
                pos_vocab.add(each[1])
        def pos_count(pos_sent, pos_vocab):
            pos_feature = [0]* len(pos_vocab)
            for each in pos_sent:
                pos_feature[pos_vocab.index(each[1])] +=1
            return pos_feature
        pos_feature_df = []
        for i in df['pos']:
            pos_feature_df.append(pos_count(i,list(pos_vocab)))
        df = pd.DataFrame(pos_feature_df, columns=list(pos_vocab))
        return df

    #########################
    
    def remove_stopword(self, x):
        dummy = []
        for i in x.split():
            if i not in self.stop_words: dummy.append(i)
        return ' '.join(dummy)
    
    def all_lower(self, x):
        return x.lower()
    def remove_punctuation(self, x):
        
        return re.sub(r'[^\w\s]', '', x)
    def remove_number(self, x):
        return ''.join([_ for _ in x if not _.isdigit()])
    def rem_un_url(self, x):
        return p.clean(x)
        
    def process(self):
        self.final_df['elogated'] = tqdm(self.x.map(self.elongated_word))
        self.final_df['emoji'] = tqdm(self.x.map(self.emoji))
        self.final_df['punctuation'] = tqdm(self.x.map(self.punctuation))
        self.final_df['allcaps'] = tqdm(self.x.map(self.all_caps))
        self.final_df['hashtag'] = tqdm(self.x.map(self.hashtag_count))
        self.final_df['negation'] = tqdm(self.x.map(self.negation))
        self.final_df['negation'] = tqdm(self.x.map(self.negation))
        
        self.final_df['new_tweets'] = tqdm(self.x.map(self.rem_un_url))
        self.final_df['new_tweets'] = tqdm(self.final_df['new_tweets'].map(self.all_lower))
        self.final_df['new_tweets'] = tqdm(self.final_df['new_tweets'].map(self.remove_number))
        
        new_df = self.pos_tag_feat(list(self.final_df['new_tweets']))
        self.final_df = pd.concat([self.final_df, new_df], axis=1)
        
        self.final_df.dropna(subset = ["new_tweets"], inplace=True)
        self.final_df['new_tweets'] = self.final_df['new_tweets'].map(self.remove_punctuation)
        
        self.final_df['new_tweets'] = self.final_df['new_tweets'].map(self.remove_stopword)
        return self

################################################################################
def printdf(df, rows = 5):
    print(df[:rows].to_markdown())
    return True 
  
#unigrams bow
def bow(X_train, X_test, X_cv, y_train, y_test, y_cv):

    count_vect = CountVectorizer() #in scikit-learn
    count_vect.fit(X_train)

    bow_counts_train = count_vect.transform(X_train)
    bow_counts_cv = count_vect.transform(X_cv)
    bow_counts_test = count_vect.transform(X_test)
    print("the shape of out text BOW vectorizer ",bow_counts_train.get_shape())
    print("the number of unique words ", bow_counts_train.get_shape()[1])
    # final_counts_test=final_counts_test.toarray()
    return bow_counts_train, bow_counts_cv, bow_counts_test

#bigrams bow
def bigram(X_train, X_test, X_cv, y_train, y_test, y_cv):
    count_vect = CountVectorizer(ngram_range=(2,2))
    bigram_counts_train = count_vect.fit_transform(X_train)
    bigram_counts_cv = count_vect.transform(X_cv)
    bigram_counts_test = count_vect.transform(X_test)
    print("some feature names ", count_vect.get_feature_names()[:10])
    print("the shape of out  BIGRAM vectorizer ",bigram_counts_train.get_shape())
    print("the number of unique words with bigrams ", bigram_counts_train.get_shape()[1])
    return bigram_counts_train, bigram_counts_cv, bigram_counts_test

In [31]:
train = pd.read_csv("data/3a/sentiment_train.csv")
test = pd.read_csv("data/3a/sentiment_test.csv")

# uncomment this later 
train = train[:300]
test = test[:200]

print(train.shape)
print(test.shape)

(300, 6)
(200, 6)


In [75]:
X_train = train["5"]
y_train = train["0"]
X_test = test["5"]
y_test = test['0']

X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

#### FEATURISATION

my_file = Path("data/3a/train_df.csv")
if my_file.is_file():
    print("CSV Found")
    train_df = pd.read_csv("data/3a/train_df.csv")
else:
    print("Generating CSV")
    #featurise train data
    proc = featurise(X_train,y_train).process()
    train_df = proc.final_df
    train_df.to_csv('data/3a/train_df.csv')

my_file = Path("data/3a/val_df.csv")
if my_file.is_file():
    print("CSV Found")
    val_df = pd.read_csv("data/3a/val_df.csv")
else:
    print("Generating CSV")
    #featurise cv data
    proc = featurise(X_cv,y_cv).process()
    val_df = proc.final_df
    val_df.to_csv('data/3a/val_df.csv')

my_file = Path("data/3a/test_df.csv")
if my_file.is_file():
    print("CSV Found")
    test_df = pd.read_csv("data/3a/test_df.csv")
else:
    print("Generating CSV")
    #featurise test data
    proc = featurise(X_test,y_test).process()
    test_df = proc.final_df
    test_df.to_csv('data/3a/test_df.csv')

Generating CSV


Generating CSV


Generating CSV


In [ ]:
test_df['target'].value_counts()

In [ ]:
# train_df.isnull().values.any()
# train_df['new_tweets'].isnull().sum()
# df['your column name'].isnull().sum()
train_df.dropna(subset = ["new_tweets"], inplace=True)
val_df.dropna(subset = ["new_tweets"], inplace=True)
test_df.dropna(subset = ["new_tweets"], inplace=True)

In [ ]:
bow_counts_train, bow_counts_cv, bow_counts_test = bow(train_df['new_tweets'], test_df['new_tweets'], val_df['new_tweets'], train_df['target'], test_df['target'], val_df['target'])
bigram_counts_train, bigram_counts_cv, bigram_counts_test = bigram(train_df['new_tweets'], test_df['new_tweets'], val_df['new_tweets'], train_df['target'], test_df['target'], val_df['target'])

In [ ]:
from scipy import sparse
def save_npz(f,name):
    sparse.save_npz("/content/drive/My Drive/IIITD Data/NLP/a3_data/npz/"+str(name)+".npz", f)
    return True

save_npz(bow_counts_cv, "bow_counts_cv")
save_npz(bow_counts_test, "bow_counts_test")
save_npz(bow_counts_train, "bow_counts_train")
save_npz(bigram_counts_train, "bigram_counts_train")
save_npz(bigram_counts_test, "bigram_counts_test")
save_npz(bigram_counts_cv, "bigram_counts_cv")

train_df.to_csv('/content/drive/My Drive/IIITD Data/NLP/a3_data/gram_traincsv.csv')
val_df.to_csv('/content/drive/My Drive/IIITD Data/NLP/a3_data/gram_valcsv.csv')
test_df.to_csv('/content/drive/My Drive/IIITD Data/NLP/a3_data/gram_testcsv.csv')

In [6]:
from nltk import pos_tag, pos_tag_sents

In [28]:
# a = df['Text'].tolist()
a= ['In situations where you wish to POS tag a column of text stored in a pandas dataframe with 1 sentence per row the majority of implementations on SO use the apply method',
    'Does that apply to this example and if so would the code be as simple as changing',
    'As mentioned in the comments',
    'but the issue is how to do this and still produce a column in a pandas dataframe?']
# comment the above portion and uncomment first line to take all tweets to list format
def pos_tag_feat(a):
    tagged_texts = pos_tag_sents(map(word_tokenize, a))

    df = pd.DataFrame(a)
    df['pos'] = tagged_texts
    pos_vocab=set()
    for pos in tagged_texts:
        for each in pos:
            pos_vocab.add(each[1])
    def pos_count(pos_sent, pos_vocab):
        pos_feature = [0]* len(pos_vocab)
        for each in pos_sent:
            pos_feature[pos_vocab.index(each[1])] +=1
        return pos_feature
    pos_feature_df = []
    for i in df['pos']:
        pos_feature_df.append(pos_count(i,list(pos_vocab)))
    df = pd.DataFrame(pos_feature_df, columns=list(pos_vocab))
    return df
pos_tag_feat(a)

,IN,PRP,TO,WRB,RB,NNP,VBP,MD,NN,WDT,NNS,DT,.,VBZ,VBN,CD,JJ,VB,VBG,CC
0,8,1,1,1,0,2,1,0,8,0,2,4,0,0,1,1,1,1,0,0
1,2,0,1,0,2,0,0,1,2,1,0,2,0,1,0,0,1,2,1,1
2,2,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0
3,1,0,1,1,1,0,0,0,4,0,0,4,1,1,0,0,0,2,0,2


# TESTING

In [36]:
pd.concat([pos_tag_feat(a),pos_tag_feat(a)], axis=1)

,IN,PRP,TO,WRB,RB,NNP,VBP,MD,NN,WDT,...,NNS,DT,.,VBZ,VBN,CD,JJ,VB,VBG,CC
0,8,1,1,1,0,2,1,0,8,0,...,2,4,0,0,1,1,1,1,0,0
1,2,0,1,0,2,0,0,1,2,1,...,0,2,0,1,0,0,1,2,1,1
2,2,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
3,1,0,1,1,1,0,0,0,4,0,...,0,4,1,1,0,0,0,2,0,2


In [ ]:
# emoji = "🤔 🙈 me así, se 😌 ds 💕👭👙 hello 👩🏾‍🎓 #emoji hello 👨‍👩‍👦‍👦 how# #are 😊 # ###you today🙅🏽🙅🏽"
# neg = "never going . to ain't be: th;e pussy, shouldn't be like doesn't is'nt?"
# general = "This is a sample sentence, showing off the stop words filtration."

In [ ]:
!pip install --user -U nltk==3.2.5 --user

In [1]:
import nltk
import sklearn

print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The nltk version is 3.2.5.
The scikit-learn version is 0.23.2.
